# 1. N-gramize 
- n그램화

## 1-a. n-gramize 계획

### -c-2. 테스트 : 1~5 gram으로 만드는 함수

In [ ]:
def generate_ngrams(tokens, max_n=5):
    """ 주어진 튜플 리스트로부터 1-gram부터 max_n-gram까지의 리스트를 생성한다. """
    ngrams_list = []
    for n in range(1, max_n + 1):
        ngrams = [' '.join([token[0] for token in tokens[i:i + n]]) for i in range(len(tokens) - n + 1)]
        ngrams_list.extend(ngrams)
    return ngrams_list


ngrams = generate_ngrams(tokens_temp)
print(ngrams)

### 4-c-3. 테스트 : 테스트용 df에 적용해보기

#### 1. 테스트 df 만들기

In [ ]:
df_test = df.head(2).copy()
df_test

In [ ]:
test_txt = df_test.iloc(0)[0]["text"]
test_txt

#### 2. 테스트 df에 테스트

In [ ]:
from ekonlpy import Mecab

# Mecab 객체 선언
mecab = Mecab(use_original_tagger=True) # set use_original_tagger=True

# generate_ngrams 함수를 적용하는 함수
def apply_ngrams(row):
    tokens = mecab.pos(row['text'])
    return generate_ngrams(tokens)

# df에 적용
df_test['ngrams'] = df_test.apply(apply_ngrams, axis=1)
df_test

#### 4. 성공 : df에 대한 테스트가 성공적으로 진행되었다.

In [ ]:
test_ngrams = df_test.iloc(0)[0]["ngrams"]
test_ngrams

## -d. 토큰화 적용
- 실제 데이터에 적용한다

### 4-d-1. df 확인

In [ ]:
df.info()

In [ ]:
df.head(2)

### 4-d-2. df에 적용

In [ ]:
from ekonlpy import Mecab

# df에 적용
df['ngrams'] = df.apply(apply_ngrams, axis=1)
df.head(3)

## 4-e. ngramize 함수화

### 4-e-1. 함수 만들기
- 논문에서는 n-gram을 5로 했지만, 차원의 저주 때문에 3으로 변경한다
- 용량이 너무 커서 작동되지 않기 때문이다.
- 그리고 4, 5-gram은 무의미한 경우가 많다.

In [ ]:
import pandas as pd
from ekonlpy import Mecab

# 시간 측정 데코레이터
import time

def time_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' executed in {end_time - start_time} seconds.")
        return result
    return wrapper

# 함수 : 토큰화 + 파일로 저장
@time_decorator
def tokenize_and_ngramize(fpath_to_read, text_column_name, fpath_to_save):
    
    # 1. 데이터 읽어오기
    print("[progress 1/5] 데이터 읽어오기")
    df = pd.read_csv(fpath_to_read, encoding="utf-8")
    
    
    # 2. n-gram 생성 함수 정의 : 주어진 튜플 리스트로부터 1-gram부터 max_n-gram까지의 리스트를 생성한다.
    print("[progress 2/5] 함수 정의 : n-gram 생성 함수")
    def generate_ngrams(tokens, max_n=3):
        ngrams_list = []
        for n in range(1, max_n + 1):
            ngrams = [' '.join([token[0] for token in tokens[i:i + n]]) for i in range(len(tokens) - n + 1)]
            ngrams_list.extend(ngrams)
        return ngrams_list

    
    # 3. 토큰화 및 n-gram 적용 함수 정의  
    print("[progress 3/5] 함수 정의 : 토큰화 및 n-gram 적용 함수")
    # mecab.pos() 는 text를 입력받아 튜플로 토큰화해주는 ekonlpy 지원 함수이다
    
    # Mecab 객체 선언
    mecab = Mecab(use_original_tagger=True) # set use_original_tagger=True
    
    def apply_ngrams(row):
        tokens = mecab.pos(row[text_column_name])
        return generate_ngrams(tokens)    


    # 4. 데이터에 적용 : 토큰화 및 n-gram화
    print("[progress 4/5] 데이터에 적용 : 토큰화 및 n-gram화")
    df['ngrams'] = df.apply(apply_ngrams, axis=1)
    

    # 5. 파일로 저장 : 토큰화 및 n-gram 적용된 데이터
    print("[progress 5/5] 파일로 저장 : 토큰화 및 n-gram 적용된 데이터")
    df_new = df.drop([text_column_name], axis=1)  # 불필요한 칼럼 제거
    df_new.to_csv(fpath_to_save, encoding='utf-8', index=False)  # csv 파일로 저장 

    print("[done]")

### 4-e-2. 테스트용 파일로 실험하기

#### 1. 테스트 .csv 파일 만들기

In [ ]:
df_test_input = df.head(3).copy()
df_test_input

In [ ]:
test_file_path = "4_test_input.csv"
df_test_input.to_csv(test_file_path, index=False, encoding="utf-8")

#### 2. 테스트 진행

In [ ]:
fpath_to_read = "4_test_input.csv"
text_column_name = "text"
fpath_to_save = "4_test_output.csv"
tokenize_and_ngramize(fpath_to_read, text_column_name, fpath_to_save)

#### 3. 테스트 결과 확인

In [ ]:
import pandas as pd
df_test_output = pd.read_csv("4_test_output.csv", encoding="utf-8")
df_test_output.head(3)

### 4-e-3. 실제 data에 적용하기

In [ ]:
fpath_to_read = "3_news_lowercase.csv"
text_column_name = "text"
fpath_to_save = "4_news_ngram.csv"
tokenize_and_ngramize(fpath_to_read, text_column_name, fpath_to_save)